In [2]:
#!/usr/bin/env python
import os
import shutil
import csv
import glob
import pandas as pd
import numpy as np

%run ../Python_files/util.py

tmc_ref_speed_dict =zload('../temp_files/tmc_ref_speed_dict.pkz')

# tmc_ref_speed_dict.keys

# AM: 7:00 am - 9:00 am
# MD: 11:00 am - 13:00 pm
# PM: 17:00 pm - 19:00 pm
# NT: 21:00 pm - 23:00 pm

data_folder = '/home/jzh/INRIX/All_INRIX_2012_filtered/'

def data_path(month):
    if month == 'Jan':
        return data_folder + 'Jan/'
    elif month == 'Apr':
        return data_folder + 'Apr/'
    elif month == 'Jul':
        return data_folder + 'Jul/'
    elif month == 'Oct':
        return data_folder + 'Oct/'
    else:
        raise IOError("Invalid input; please input another month.")
        

def create_HDFS(month):
    data_dir = data_path(month)
    for input_file in glob.glob(data_dir + '*.csv'):
        with open(input_file, 'rb') as inp, \
                open(input_file + '.add_first_row', 'wb') as out:
            writer = csv.writer(out)
            row = ['tmc'] + ['month'] + ['day'] + ['dow'] + ['hour'] + \
                  ['minute'] + ['speed'] + ['avg_speed'] + ['ref_speed'] + \
                  ['travel_time'] + ['confidence_score','c_value']
            writer.writerow(row)
            for row in csv.reader(inp):
                if 'tmc' not in row:
                    writer.writerow(row)

                
    # cf. http://world77.blog.51cto.com/414605/552326
    # check if path exists
    if os.path.isdir(data_dir):    
        print ("Directory exists")
    else:
        print ("Directory does not exist; please input right dir") 

    filelist = []

    filelist = os.listdir(data_dir)    # obtain file names

    # print filelist

    for i in filelist:
        NewFile = i.replace("csv.add_first_row", "csv")   
        # print NewFile     
        shutil.move(data_dir + i, data_dir + NewFile)   

    df_list = []

    for input_file in glob.glob(data_dir + '*.csv'):
        df_list.append(pd.read_csv(input_file, dtype={'tmc':str, 'month':np.int64, 
                                                      'day':np.int64, 'dow':np.int64, 'hour':np.int64, 
                                                      'minute':np.int64, 'speed':np.float64, 
                                                      'avg_speed':np.float64, 'ref_speed':np.float64, 
                                                      'travel_time':np.float64, 'confidence_score':str, 
                                                      'c_value':str}))

    df = pd.concat(df_list)

    store = pd.HDFStore(data_folder + 'store_df_%s.h5' %month) 

    del df['avg_speed']
    del df['ref_speed']
    del df['confidence_score']
    del df['c_value']

    store['df_%s' %month] = df  # save result
    
create_HDFS('Jan')
create_HDFS('Apr')
create_HDFS('Jul')
create_HDFS('Oct')

Directory exists
Directory exists
Directory exists
Directory exists
